In [2]:
###   importing all libraries

from sklearn import datasets
from nltk.corpus import stopwords
import random
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag
import pickle

In [3]:
###  making list of stopwords aand punctuators
stopWords = stopwords.words('english')
punct=list(string.punctuation)
lemm = WordNetLemmatizer()
stopWords=stopWords+punct

In [4]:
### fetching datasets
data=datasets.fetch_20newsgroups()


In [5]:
###  finding location till where data is useless

def no_of_lines(file1):
    """THIS FUNCTION RETURN THE NO OF REQUIRED LINES..
        IT HELPS TO REMOVE THE HEADER PART"""
    result=15
    l1=[]
    for h in file1:
        h=h.split(" ")
        if(len(h)>0):
            line=h[0]
            if line=='Lines:':
                
                try:
                    result=(h[1])
                    break;
                except:
                    result=15
            
    return (result)
        
    

In [6]:
### finding POS for lemmetization

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return ''

In [7]:
### finding the root words

def lemmatization(pos):
        l1=[]
        for word in pos:
            #print(type(word))
            word=list(word)
            a=get_wordnet_pos(word[1])
            try:
                new_word=lemm.lemmatize(word[0],pos=a)
                l1.append(new_word)
            except:
                l1.append(word[0])
        return l1   

In [98]:
#### cleaning words

def all_words(doc):
    l1=(', '.join(doc))
    l1=(word_tokenize(l1))
    final_doc=[]
    for l in l1:
        l=''.join([i for i in l if not i.isdigit()])
        for c in string.punctuation:
            l=l.replace(c,"")
        if len(l)>2:
            if l.lower() not in stopWords:
                 if not l.endswith("edu"):
                           final_doc.append(l.lower())
    return(final_doc)
   

In [99]:
def document(train):
    documents=[]
    d=((train).split('\n'))
    try:
        result=int(no_of_lines(d))
    except:
        result=15
    
    lines=d[-result:]
    docs=all_words(lines)
    #print(docs)
       
    pos=(pos_tag(docs))
    #print(pos)
        
    document=lemmatization(pos)
    
    for w in document:
        if w not in stopWords:
            documents.append(w)
        else:
            continue
    return documents
    

In [112]:
### creating data and target values

text_documents=[]
target_documents=[]
for i in range(len(data.data)):
    print(i)
    text_documents.append(" ".join(document(data.data[i])))
    target_documents.append(data.target[i])


In [42]:
### import countvectorizer

from sklearn.feature_extraction.text import CountVectorizer

In [101]:
### split data into train and test

from sklearn.model_selection import train_test_split
trainX,testX,trainY,testY=train_test_split(text_documents,target_documents,random_state=1,test_size=0.2)

In [105]:
count_vex=CountVectorizer(max_features=30000)
train_feature=count_vex.fit_transform(trainX)
test_feature=count_vex.transform(testX)

In [106]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(train_feature,trainY)
lr.score(test_feature,testY)

0.8161732213875387